# Import

In [77]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score, classification_report, make_scorer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder, Normalizer, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split, cross_validate, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier
import seaborn as sns
import datetime
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os

pd.set_option('display.max_rows', 500)
random_state = 42
np.random.seed(random_state)
# data_filepath = Path('./data')
data_filepath = Path('/kaggle/input/child-mind-institute-problematic-internet-use')
KAPPA_SCORER = make_scorer(
    cohen_kappa_score, 
    greater_is_better=True, 
    weights='quadratic',
)

# Data

In [41]:
!du -hs $data_filepath/*
train_df = pd.read_csv(data_filepath / 'train.csv')
test_df = pd.read_csv(data_filepath / 'test.csv')
train_df.shape, test_df.shape

12K	/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv
4.0K	/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv
8.0M	/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet
6.3G	/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet
8.0K	/kaggle/input/child-mind-institute-problematic-internet-use/test.csv
924K	/kaggle/input/child-mind-institute-problematic-internet-use/train.csv


((3960, 82), (20, 59))

In [51]:
parquet_partitions = list((data_filepath / 'series_train.parquet').glob('id=*'))
len(parquet_partitions), len(set(str(c).split('=')[1] for c in parquet_partitions) & set(train_df.id))

(996, 996)

In [52]:
%%time
parquet_filepath = parquet_partitions[0]
print(f'{parquet_filepath=!s}')
train_ts = pd.read_parquet(parquet_filepath)
train_ts.info()

parquet_filepath=/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=0745c390
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50458 entries, 0 to 50457
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   step                 50458 non-null  uint32 
 1   X                    50458 non-null  float32
 2   Y                    50458 non-null  float32
 3   Z                    50458 non-null  float32
 4   enmo                 50458 non-null  float32
 5   anglez               50458 non-null  float32
 6   non-wear_flag        50458 non-null  float32
 7   light                50458 non-null  float32
 8   battery_voltage      50458 non-null  float32
 9   time_of_day          50458 non-null  int64  
 10  weekday              50458 non-null  int8   
 11  quarter              50458 non-null  int8   
 12  relative_date_PCIAT  50458 non-null  float32
dtypes: float32(9), int64(1), i

In [94]:
def parse_time_of_day(nanoseconds):
    seconds = nanoseconds // 10**9
    nanoseconds_remainder = nanoseconds % 10**9
    time_of_day = datetime.timedelta(seconds=seconds)
    hours, remainder = divmod(time_of_day.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
#     formatted_time = f"{hours:02}:{minutes:02}:{seconds:02}.{nanoseconds_remainder:09}"
    return hours, minutes, seconds, nanoseconds_remainder

train_ts['day_hour'] = train_ts.time_of_day.apply(lambda x: parse_time_of_day(x)[0])
train_ts['week'] = train_ts['relative_date_PCIAT'] // 7 + 1
train_ts['month'] = train_ts['relative_date_PCIAT'] // 30 + 1

In [ ]:
agg_funcs = ['count', 'mean', 'std', 'min', 'max']
group_cols = ['']

In [73]:
train_ts.describe()

,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
count,50458.000000,50458.000000,50458.000000,50458.000000,50458.000000,50458.000000,50458.0,50458.000000,50458.000000,5.045800e+04,50458.000000,50458.0,50458.000000
mean,25228.500000,-0.054638,-0.163923,-0.114302,0.045252,-7.805897,0.0,46.009533,4027.514893,5.415475e+13,4.438860,2.0,30.202068
std,14566.114278,0.633126,0.513286,0.500372,0.132576,34.917873,0.0,205.862213,108.451317,1.876976e+13,1.825557,0.0,11.773107
min,0.000000,-1.812031,-2.631380,-1.798073,0.000000,-89.987045,0.0,0.000000,3829.000000,0.000000e+00,1.000000,2.0,15.000000
25%,12614.250000,-0.701660,-0.619076,-0.536432,0.007953,-32.948602,0.0,2.520257,3958.000000,4.325125e+13,3.000000,2.0,17.000000
50%,25228.500000,0.015846,-0.141810,-0.104193,0.019257,-6.358004,0.0,8.230733,4029.000000,5.630500e+13,5.000000,2.0,28.000000
75%,37842.750000,0.437897,0.148919,0.223770,0.036048,13.095750,0.0,24.750000,4146.000000,6.978000e+13,6.000000,2.0,38.000000
max,50457.000000,1.850391,3.580182,1.738203,5.314874,89.422226,0.0,2626.199951,4187.000000,8.639500e+13,7.000000,2.0,57.000000
